info on Cartopy https://scitools.org.uk/cartopy/docs/latest/
build requirements to use Cartopy https://github.com/SciTools/cartopy/blob/master/INSTALL ** conda install -c conda-forge cartopy ** That should work

### NOTE:

I worked with Michael to update the system so all these required installs should be added already. I am including this information though, for reference in case we need to use these notebooks on our own computers and not the hub.

___

In [ ]:
#conda install -c conda-forge cartopy 

# Groundtracking
## Modeling a Satellite in Orbit

___

Below we import the necessary packages to run the groundtracking model

A useful website
https://www.celestrak.com/NORAD/documentation/tle-fmt.php

### Below, we input the standard TLE fromat to describe the orbit of the Satellite

The website linked has varying satellites collected. I am working on inputing our own without creating a website to input the TLE format.

In [1]:
import matplotlib.pyplot as plt
plt.ion() # Turning interactive matplotlib on

# SEE ABOVE for downloading to use
import cartopy.crs as ccrs 
from orbit_predictor.sources import get_predictor_from_tle_lines
from orbit_predictor.sources import NoradTLESource
from orbit_predictor.locations import Location

import datetime as dt
import pandas as pd

In [2]:
# TLE INPUT FILE WITH MANY SATELLITES
source = NoradTLESource.from_url("http://www.celestrak.com/NORAD/elements/resource.txt")
predictor = source.get_predictor("NUSAT-2")
predictor

And below is how we input our own TLE for groundtracking.

___

In [3]:
TLE_LINES = ('1 40014U 14033E   15328.83189478  .00002037  00000-0  19542-3 0  9993', 
             '2 40014  97.9563 235.2185 0033978  56.8458 303.6015 14.93852659 78029')

predictor = get_predictor_from_tle_lines(TLE_LINES)

predictor.get_position(dt.datetime(2019, 10, 14))

Position(when_utc=datetime.datetime(2019, 10, 14, 0, 0), position_ecef=(-1210.125549520668, -2032.2427564825334, -6545.272705228332), velocity_ecef=(-5.959942655730757, -3.992472731370765, 2.366305293077898), error_estimate=None)

Below is a neat extra function. Useless for our purposes but if you care to know when the next time a satellite will cross any specific point the code block below will do this.

CU Denvers lat and lon is 
39° 44' 31.3548'' N
104° 59' 29.5116'' W
lat = 39.742043
long = -104.991531

This is the lat / lon of the Student Commons Building
39.745830
-105.003193

The input is Latitude, Longitude, Altitude.

In [4]:
#FBR = Location("Observatori Fabra", 41.4184, 2.1239, 408)
FBR = Location("CU Denver", 39.745830, -105.003193, 1590)

predicted_pass = predictor.get_next_pass(FBR)
predicted_pass

<PredictedPass 40014 over CU Denver on 2019-10-17 19:40:22.097145>

___

Documentation on the Pandas Date_range
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html


From the website above:

*Of the four parameters start, end, periods, and freq, exactly three must be specified. If freq is omitted, the resulting DatetimeIndex will have periods linearly spaced elements between start and end (closed on both sides).*

I will be trying a few varieties

In [5]:
dates = pd.date_range(start="2019-10-09 00:00", end="2019-10-10 00:00", periods = 50)

In [6]:
pd.date_range(start='10/14/2019', end='10/15/2019')

DatetimeIndex(['2019-10-14', '2019-10-15'], dtype='datetime64[ns]', freq='D')

In [7]:
pd.date_range(start='10/14/2019', periods=8)

DatetimeIndex(['2019-10-14', '2019-10-15', '2019-10-16', '2019-10-17',
               '2019-10-18', '2019-10-19', '2019-10-20', '2019-10-21'],
              dtype='datetime64[ns]', freq='D')

In [8]:
pd.date_range(end='10/15/2019', periods=8)

DatetimeIndex(['2019-10-08', '2019-10-09', '2019-10-10', '2019-10-11',
               '2019-10-12', '2019-10-13', '2019-10-14', '2019-10-15'],
              dtype='datetime64[ns]', freq='D')

In [9]:
pd.date_range(start='2019-10-14', end='2019-10-15', periods=3)

DatetimeIndex(['2019-10-14 00:00:00', '2019-10-14 12:00:00',
               '2019-10-15 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [10]:
pd.date_range(start='10/14/2019', periods=5, freq='M')

DatetimeIndex(['2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31',
               '2020-02-29'],
              dtype='datetime64[ns]', freq='M')

In [11]:
pd.date_range(end='10/15/2019', periods=5, freq='3M')

DatetimeIndex(['2018-09-30', '2018-12-31', '2019-03-31', '2019-06-30',
               '2019-09-30'],
              dtype='datetime64[ns]', freq='3M')

Still trying to decipher what is best above

___

Below, hmm what is this. Will have to document later because I am out of time right now

In [12]:
latlon = pd.DataFrame( index = dates, columns = ["lat", "lon"] )

for date in dates:
    lat, lon, _ = predictor.get_position( date ).position_llh
    latlon.loc[ date ] = ( lat, lon )

In [ ]:
plt.figure( figsize = ( 20, 30 ) )
ax = plt.axes( projection = ccrs.PlateCarree() )
ax.stock_img()

plt.plot( latlon["lon"], latlon["lat"], 'k',
         transform = ccrs.Geodetic() );